In [3]:
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import pymysql

db = pymysql.connect(host="localhost", user="rnv_importer", password="rnv_importer", database="rnv_big_data_mining")
cursor = db.cursor()

cursor.execute("SELECT id from journeys;")
journeys = cursor.fetchall()

station =[]
delay = []
stopAmount = []
averageDelay = []

for journey in journeys:
    cursor.execute("SELECT api_station, api_plannedDeparture, api_realtimeDeparture from stops where api_journey = %s ORDER BY api_plannedDeparture ASC;", journey)
    stops = cursor.fetchall()
    if stops:
        list(stops).pop(0) #has value null because of last stop not having a departure
        laststops_rt_dep = None
        for stop in stops:
            
            if stop[1] is None or stop[2] is None:
                break                
            station_id = stop[0]
            planned_departure = stop[1].timestamp()
            realtime_departure = stop[2].timestamp()

            # Füge die Station hinzu oder aktualisiert sie
            if station_id in station:
                # Station existiert bereits, aktualisiere die Werte
                index = station.index(station_id)
                if laststops_rt_dep is None:
                    delay[index] += (realtime_departure - planned_departure)
                else:
                    delay[index] += (realtime_departure - laststops_rt_dep)
                stopAmount[index] += 1
            else:
                # Station existiert noch nicht, füge sie hinzu
                station.append(station_id)
                if laststops_rt_dep is None:
                    delay.append(realtime_departure - planned_departure)
                else:
                    delay.append(realtime_departure - laststops_rt_dep)
                stopAmount.append(1)
            laststops_rt_dep = realtime_departure
    
for station_id in station:
    index = station.index(station_id)

    total_delays = delay[index]
    total_stops = stopAmount[index]
    average_delay = total_delays / total_stops
    averageDelay.append(average_delay)
    print(f"Station: {station_id}, stops: {total_stops}, durchschnittliche Verspätung: {average_delay}, gesamte Verspätung(absolut): {total_delays}")

Station: 1457, stops: 416, durchschnittliche Verspätung: 14.745192307692308, gesamte Verspätung(absolut): 6134.0
Station: 1308, stops: 1, durchschnittliche Verspätung: 0.0, gesamte Verspätung(absolut): 0.0
Station: 1320, stops: 1, durchschnittliche Verspätung: 120.0, gesamte Verspätung(absolut): 120.0
Station: 1325, stops: 1, durchschnittliche Verspätung: 60.0, gesamte Verspätung(absolut): 60.0
Station: 1317, stops: 1, durchschnittliche Verspätung: 60.0, gesamte Verspätung(absolut): 60.0
Station: 1309, stops: 1, durchschnittliche Verspätung: 60.0, gesamte Verspätung(absolut): 60.0
Station: 1321, stops: 1, durchschnittliche Verspätung: 60.0, gesamte Verspätung(absolut): 60.0
Station: 1328, stops: 1, durchschnittliche Verspätung: 60.0, gesamte Verspätung(absolut): 60.0
Station: 1323, stops: 1, durchschnittliche Verspätung: 120.0, gesamte Verspätung(absolut): 120.0
Station: 1315, stops: 1, durchschnittliche Verspätung: 180.0, gesamte Verspätung(absolut): 180.0
Station: 1287, stops: 1, dur